# Lab 4.1: Implementing Sentiment Analysis
 


In this lab, you will develop a solution to perform sentiment analysis on the Internet Movie Database (IMDB) dataset.

## Learning objectives

- Evaluate machine learning (ML) algorithms that are used in natural language processing (NLP) for sentiment analysis
- Create a solution to a sentiment analysis business problem.

## Introducing the business scenario

In this lab, you will play the role of a data scientist on a small development team. The organization that you work for maintains a website of movie reviews. A key customer feature was identified: to provide an overall *Smiley Face* (positive inference) or *Sad Face* (negative inference) for a particular movie based on the number of its positive and negative reviews. You will develop an ML solution that developers can use to create an inference for a movie review. You will need to analyze the review and indicate if it is positive or negative.

To help with this task, you have access to a dataset that contains the raw text of 50,000 movie reviews. These reviews have been labeled either as positive or negative.

About this dataset
The Large Movie Review Dataset is a collection of highly polar movie reviews. This data supports work in the following paper:

Andrew L. Maas, Raymond E. Daly, Peter T. Pham, Dan Huang, Andrew Y. Ng, and Christopher Potts. "Learning Word Vectors for Sentiment Analysis." Presented at the 49th Annual Meeting of the Association for Computational Linguistics (ACL 2011), Portland, Oregon, USA, June 2011. http://ai.stanford.edu/~amaas/data/sentiment/.

The dataset contains a single text field containing the review. The dataset is labeled either positive (1) or negative (0).

The dataset contains the following features:

text: Text of the review
label: Whether the review is positive or negative (1 or 0)

## Lab steps

To complete this lab, you will follow these steps:

1. [Installing packages](#1.-Installing-packages)
2. [Reading the dataset](#2.-Reading-the-dataset)
3. [Performing exploratory data analysis](#3.-Performing-exploratory-data-analysis)
4. [Running the first pass: Minimal processing](#4.-Running-the-first-pass:-Minimal-processing)
5. [Running the second pass: Normalizing the text](#5.-Running-the-second-pass:-Normalizing-the-text)
6. [Tuning hyperparameters](#6.-Tuning-hyperparameters)
7. [Using BlazingText](#7.-Using-BlazingText)
8. [Using-Amazon Comprehend](#8.-Using-Amazon-Comprehend)

## Submitting your work

1. In the lab console, choose **Submit** to record your progress and when prompted, choose **Yes**.

1. If the results don't display after a couple of minutes, return to the top of the lab instructions and choose **Grades**.

**Tip:** You can submit your work multiple times. After you change your work, choose **Submit** again. Your last submission is what will be recorded for this lab.

1. To find detailed feedback on your work, choose **Details** followed by **View Submission Report**.

## 1. Installing packages
([Go to top](#Lab-4.1:-Implementing-Sentiment-Analysis))

Start by updating and installing the packages that you will use in the notebook. 


In [ ]:
#Install/Upgrade dependencies
!pip install --upgrade pip
!pip install --upgrade scikit-learn
!pip install --upgrade sagemaker
!pip install --upgrade nltk
!pip install --upgrade seaborn

__Note:__ Before you proceed with this lab for the first time, we recommend that you restart the kernel by choosing __Kernel__ > __Restart Kernel__.

Import the packages that are used in the notebook.

In [ ]:
import boto3
import os, io, struct
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score, roc_curve, auc, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

The following code cell includes a few helper functions that  plot a confusion matrix and calculate other key metrics.

In [ ]:

def plot_confusion_matrix(test_labels, target_predicted):
    matrix = confusion_matrix(test_labels, target_predicted)
    df_confusion = pd.DataFrame(matrix)
    colormap = sns.color_palette("BrBG", 10)
    sns.heatmap(df_confusion, annot=True, fmt='.2f', cbar=None, cmap=colormap)
    plt.title("Confusion Matrix")
    plt.tight_layout()
    plt.ylabel("True Class")
    plt.xlabel("Predicted Class")
    plt.show()
    
def print_metrics(test_labels, target_predicted_binary):
    TN, FP, FN, TP = confusion_matrix(test_labels, target_predicted_binary).ravel()
    # Sensitivity, hit rate, recall, or true positive rate
    Sensitivity  = float(TP)/(TP+FN)*100
    # Specificity or true negative rate
    Specificity  = float(TN)/(TN+FP)*100
    # Precision or positive predictive value
    Precision = float(TP)/(TP+FP)*100
    # Negative predictive value
    NPV = float(TN)/(TN+FN)*100
    # Fall out or false positive rate
    FPR = float(FP)/(FP+TN)*100
    # False negative rate
    FNR = float(FN)/(TP+FN)*100
    # False discovery rate
    FDR = float(FP)/(TP+FP)*100
    # Overall accuracy
    ACC = float(TP+TN)/(TP+FP+FN+TN)*100

    print(f"Sensitivity or TPR: {Sensitivity}%")    
    print(f"Specificity or TNR: {Specificity}%") 
    print(f"Precision: {Precision}%")   
    print(f"Negative Predictive Value: {NPV}%")  
    print( f"False Positive Rate: {FPR}%") 
    print(f"False Negative Rate: {FNR}%")  
    print(f"False Discovery Rate: {FDR}%" )
    print(f"Accuracy: {ACC}%") 



## 2. Reading the dataset

([Go to top](#Lab-4.1:-Implementing-Sentiment-Analysis))

In this section, you will load the dataset. The dataset has already been downloaded by Amazon Sagemaker Studio. Use the __pandas__ library to read the dataset.

#### __Loading the training data:__

In [ ]:
df = pd.read_csv('../data/imdb.csv', header=0)

## 3. Performing exploratory data analysis
([Go to top](#Lab-4.1:-Implementing-Sentiment-Analysis))

In this section, you will examine the dataset. 

Complete the following functions. The first one has been provided so that you can learn the format.


### Challenge: Listing the first eight rows

In [ ]:
def show_eight_rows(df):
    # Implement this function
    return df.head(8)    

In [ ]:
print(show_eight_rows(df))

### Challenge: What is the shape of the data?

In [ ]:
def show_data_shape(df):
    # Implement this function
    ### BEGIN_SOLUTION
    return df.shape
    ### END_SOLUTION

In [ ]:
print(show_data_shape(df))

### Challenge: How many positive and negative instances are in the data?

In [ ]:
def show_data_instances(df):
    # Implement this function
    ### BEGIN_SOLUTION
    return df['label'].value_counts()
    ### END_SOLUTION

In [ ]:
print(show_data_instances(df))

### Challenge: Does the data have any missing values?

In [ ]:
def show_missing_values(df):
    # Implement this function
    ### BEGIN_SOLUTION
    return df.isna().sum()
    ### END_SOLUTION
    

In [ ]:
print(show_missing_values(df))

## 4. Running the first pass: Minimal processing
([Go to top](#Lab-4.1:-Implementing-Sentiment-Analysis))

In this section, you will perform the minimum steps that are needed to train a classification model. You will use this trained model to see the impact of processing text on the results.

Start by importing the Natural Langauge ToolKit (NLTK) package and the Regular Expression (re) package.

In [ ]:
import nltk, re
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize

### Challenge: Splitting the data into datasets for training, validation, and testing

In this task, you will split the dataset so that you have 80 percent of the dataset for training, and 10 percent each for validation and testing.

To split the dataset, use the `train_test_split` function from __scikit-learn__. For more information about this function, see the [scikit-learn test_train_split documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) at https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html.

Specify __df__ as the dataset. Split this dataset into a __train__ set and a __test_and_validate__ set. Then, split the __train_and_validate__ set into the __test__ set and the __validate__ set.

(*Optional*) For repeatable results, shuffle the deck and **random_state**.

In [ ]:
from sklearn.model_selection import train_test_split
# uncomment the following lines and implement your solution
def split_data(df):
    # train, test_and_validate = train_test_split(....)
    # test, validate = train_test_split(....)
    ### BEGIN_SOLUTION
    train, test_and_validate = train_test_split(df,
                                            test_size=0.2,
                                            shuffle=True,
                                            random_state=324
                                            )
    test, validate = train_test_split(test_and_validate,
                                                test_size=0.5,
                                                shuffle=True,
                                                random_state=324)
    ### END_SOLUTION
    return train, validate, test

Check whether your datasets are split correctly by running the following code cell.

In [ ]:
train, validate, test = split_data(df)
print(train.shape)
print(test.shape)
print(validate.shape)

### Assembling the processing pipeline

In this cell, the basic processing pipeline is assembled for the text data. You will now modify this implementation to add more features.

In [ ]:
%%time

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

text_features = ['text']
model_target = 'label'

text_processor_0 = Pipeline([
    ('text_vect_0', CountVectorizer(max_features=500))
])

data_preprocessor = ColumnTransformer([
    ('text_pre_0', text_processor_0, text_features[0])
])

print('Datasets shapes before processing: ', train.shape, validate.shape, test.shape)
train_matrix = data_preprocessor.fit_transform(train)
test_matrix = data_preprocessor.transform(test)
validate_matrix = data_preprocessor.transform(validate)
print('Datasets shapes after processing: ', train_matrix.shape, validate_matrix.shape, test_matrix.shape)

To train a model, the data must be uploaded to Amazon Simple Storage Service (Amazon S3) in the correct format. XGBoost uses a comma-separated values (CSV) file.

In [ ]:
s3_resource = boto3.Session().resource('s3')

def upload_s3_csv(filename, folder, X_train, y_train, is_test=False):
    csv_buffer = io.StringIO()
    features = [t.toarray().astype('float32').flatten().tolist() for t in X_train]
    if is_test:
        temp_list = features
    else:
        temp_list = np.insert(features, 0, y_train['label'], axis=1)
    np.savetxt(csv_buffer, temp_list, delimiter=',' )
    s3_resource.Bucket(bucket).Object(os.path.join(prefix, folder, filename)).put(Body=csv_buffer.getvalue())

In [ ]:
bucket = 'c51302a798363l1767466t1w753256443787-labbucket-vzr6xg8irt81'

Set the file names for this pass.

In [ ]:
prefix='lab41'
train_file='train-pass1.csv'
validate_file='validate-pass1.csv'
test_file='test-pass1.csv'

Upload the train, validate, and test datasets to Amazon S3.

In [ ]:
upload_s3_csv(train_file, 'train', train_matrix, train)
upload_s3_csv(validate_file, 'validate', validate_matrix, validate)
upload_s3_csv(test_file, 'test', test_matrix, test, True)

### Challenge: Training an XGBoost model

Uncomment and complete the following SageMaker function to create an `Estimator`. Use the following parameters:
- **role**: Use the current SageMaker role (__Hint:__ Use `sagemaker.get_execution_role()`)
- **instance count**: `1`
- **instance type**: `ml.m5.xlarge`


In [ ]:
import sagemaker
from sagemaker.image_uris import retrieve
container = retrieve('xgboost',boto3.Session().region_name,'1.0-1')
s3_output_location=f's3://{bucket}/{prefix}/output/'

hyperparams={"num_round":"42",
             "eval_metric": "error",
             "objective": "binary:logistic",
             "silent" : 1}

# xgb_model=sagemaker.estimator.Estimator(container,
#                                         role=<INSERT_ROLE_HERE>,
#                                         instance_count=<INSERT_COUNT_HERE>,
#                                         instance_type=<INSERT_INSTANCE_TYPE_HERE>,
#                                         output_path=s3_output_location,
#                                         hyperparameters=hyperparams,
#                                         sagemaker_session=sagemaker.Session())
### BEGIN_SOLUTION
xgb_model=sagemaker.estimator.Estimator(container,
                                        role=sagemaker.get_execution_role(),
                                        instance_count=1,
                                        instance_type='ml.m5.2xlarge',
                                        output_path=s3_output_location,
                                        hyperparameters=hyperparams,
                                        sagemaker_session=sagemaker.Session())
### END_SOLUTION

Set up the two data channels. One data channel is for the training data that's used to train the model. The second data channel is for the validation data that's used to generate performance metrics.

In [ ]:
train_channel = sagemaker.inputs.TrainingInput(
    f's3://{bucket}/{prefix}/train/{train_file}',
    content_type='text/csv')

validate_channel = sagemaker.inputs.TrainingInput(
    f's3://{bucket}/{prefix}/validate/{validate_file}',
    content_type='text/csv')

data_channels = {'train': train_channel, 'validation': validate_channel}

Train the model. (This step might take a few minutes.)

In [ ]:
%%time

xgb_model.fit(inputs=data_channels, logs=False, job_name='xgb-pass1-'+datetime.now().strftime("%m-%d-%Y-%H-%M-%S"))

Display the metrics from the current XGBoost job.

In [ ]:
sagemaker.analytics.TrainingJobAnalytics(xgb_model._current_job_name, 
                                         metric_names = ['train:error','validation:error']
                                        ).dataframe()

The initial results don't seem to be useful. Use the __test__ dataset to calculate more metrics. (This step might take a few minutes.)

In [ ]:
%%time

upload_s3_csv('batch-in.csv', 'batch-in', test_matrix, test, True)
batch_X_file='batch-in.csv'
batch_output = f's3://{bucket}/{prefix}/batch-out/'
batch_input = f's3://{bucket}/{prefix}/batch-in/{batch_X_file}'

xgb_transformer = xgb_model.transformer(instance_count=1,
                                       instance_type='ml.m5.2xlarge',
                                       strategy='MultiRecord',
                                       assemble_with='Line',
                                       output_path=batch_output)

xgb_transformer.transform(data=batch_input,
                         data_type='S3Prefix',
                         content_type='text/csv',
                         split_type='Line',
                         job_name='xgboost-pass1')
xgb_transformer.wait(logs=False)

In [ ]:
s3 = boto3.client('s3')
obj = s3.get_object(Bucket=bucket, Key=f'{prefix}/batch-out/batch-in.csv.out')
target_predicted = pd.read_csv(io.BytesIO(obj['Body'].read()),',',names=['class'])

def binary_convert(x):
    threshold = 0.5
    if x > threshold:
        return 1
    else:
        return 0

target_predicted_binary = target_predicted['class'].apply(binary_convert)


In [ ]:
plot_confusion_matrix(test['label'], target_predicted_binary)

In [ ]:
print_metrics(test['label'], target_predicted_binary)

## 5. Second pass: Normalizing the text
([Go to top](#Lab-4.1:-Implementing-Sentiment-Analysis))

In this section, you will perform some standard preprocessing tasks on the text before you retrain the model.

### Challenge: Removing stopwords that might impact sentiment

You could remove all the stopwords, but you might want to keep the stopwords that could impact the sentiment, such as __not__ or __don't__. 

A few stopwords to exclude have been provided. Update the function to remove other words that might impact sentiment.

In [ ]:
# Get a list of stopwords from the NLTK library
stop = stopwords.words('english')

def remove_stopwords(stopwords):
    # Implement this function
    excluding = ['against', 'not', 'don', 'don\'t','ain', 'are', 'aren\'t']
    ### BEGIN_SOLUTION
    excluding = ['against', 'not', 'don', 'don\'t','ain', 'are', 'aren\'t', 'could', 'couldn\'t',
             'did', 'didn\'t', 'does', 'doesn\'t', 'had', 'hadn\'t', 'has', 'hasn\'t', 
             'have', 'haven\'t', 'is', 'isn\'t', 'might', 'mightn\'t', 'must', 'mustn\'t',
             'need', 'needn\'t','should', 'shouldn\'t', 'was', 'wasn\'t', 'were', 
             'weren\'t', 'won\'t', 'would', 'wouldn\'t']
    ### END_SOLUTION
    return [word for word in stop if word not in excluding]

# New stopword list
stopwords = remove_stopwords(stop)


### Challenge: Adding cleanup steps

Update the following `clean` function to complete the following tasks:
- Remove leading spaces and trailing spaces
- Remove any HTML tags


In [ ]:
snow = SnowballStemmer('english')
def clean(sent):
    # Implement this function
    sent = sent.lower()
    sent = re.sub('\s+', ' ', sent)
    ### BEGIN_SOLUTION
    sent = sent.strip()
    sent = re.compile('<.*?>').sub('',sent)
    ### END_SOLUTION
    filtered_sentence = []
    
    for w in word_tokenize(sent):
        # You are applying custom filtering here. Feel free to try different things.
        # Check if it is not numeric, its length > 2, and it is not in stopwords
        if(not w.isnumeric()) and (len(w)>2) and (w not in stopwords):  
            # Stem and add to filtered list
            filtered_sentence.append(snow.stem(w))
    final_string = " ".join(filtered_sentence) #final string of cleaned words
    return final_string

Create new test, validate, and test dataframes by using the function that you created earlier.

In [ ]:
# Uncomment the next line and implement the function call to split_data
#train, validate, test = 

### BEGIN_SOLUTION
train, validate, test = split_data(df)
### END_SOLUTION

print(train.shape)
print(test.shape)
print(validate.shape)

The pipeline has been updated to include a call to the previously defined `clean` function from the `CountVectorizer`. This function will take a little longer to run.

In [ ]:
%%time

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

text_features = ['text']
model_target = 'label'

text_processor_0 = Pipeline([
    ('text_vect_0', CountVectorizer(preprocessor=clean, max_features=500))
])

data_preprocessor = ColumnTransformer([
    ('text_pre_0', text_processor_0, text_features[0])
])

print('Datasets shapes before processing: ', train.shape, validate.shape, test.shape)
train_matrix = data_preprocessor.fit_transform(train)
test_matrix = data_preprocessor.transform(test)
validate_matrix = data_preprocessor.transform(validate)
print('Datasets shapes after processing: ', train_matrix.shape, validate_matrix.shape, test_matrix.shape)

Set the file names for this pass.

In [ ]:
prefix='lab41'
train_file='train_pass2.csv'
validate_file='validate_pass2.csv'
test_file='test_pass2.csv'

### Challenge: Uploading the files to Amazon S3

Use the previous code to upload the new files to Amazon S3. 

__Tip:__ Copy the code and paste it into the following code cell.

In [ ]:
### BEGIN_SOLUTION
upload_s3_csv(train_file, 'train', train_matrix, train)
upload_s3_csv(validate_file, 'validate', validate_matrix, validate)
upload_s3_csv(test_file, 'test', test_matrix, test, True)
### END_SOLUTION

### Challenge: Creating the estimator and setting up the data channels

Use the previous code to set up the estimator and data channels. 

__Tip:__ Copy the code from the previous cell and paste it into the following cell.

In [ ]:
%%time

container = retrieve('xgboost',boto3.Session().region_name,'1.0-1')

hyperparams={"num_round":"42",
             "eval_metric": "error",
             "objective": "binary:logistic",
             "silent" : 1}

### BEGIN_SOLUTION
xgb_model=sagemaker.estimator.Estimator(container,
                                        sagemaker.get_execution_role(),
                                        instance_count=1,
                                        instance_type='ml.m5.2xlarge',
                                        output_path=s3_output_location,
                                        hyperparameters = hyperparams,
                                        sagemaker_session=sagemaker.Session())

train_channel = sagemaker.inputs.TrainingInput(
    f's3://{bucket}/{prefix}/train/{train_file}',
    content_type='text/csv')

validate_channel = sagemaker.inputs.TrainingInput(
    f's3://{bucket}/{prefix}/validate/{validate_file}',
    content_type='text/csv')

data_channels = {'train': train_channel, 'validation': validate_channel}

### END_SOLUTION

xgb_model.fit(inputs=data_channels, logs=False, job_name='xgb-pass2-'+datetime.now().strftime("%m-%d-%Y-%H-%M-%S"))

In [ ]:
sagemaker.analytics.TrainingJobAnalytics(xgb_model._current_job_name, 
                                         metric_names = ['train:error','validation:error']
                                        ).dataframe()

### Challenge: Creating a batch transformer job

Using the previous code, create a transformer job. (This step might take a few minutes to complete.) 

__Tip:__ Copy the code from the previous example and paste it into the following cell.

In [ ]:
%%time

### BEGIN_SOLUTION
xgb_transformer = xgb_model.transformer(instance_count=1,
                                       instance_type='ml.m5.2xlarge',
                                       strategy='MultiRecord',
                                       assemble_with='Line',
                                       output_path=batch_output)

xgb_transformer.transform(data=batch_input,
                         data_type='S3Prefix',
                         content_type='text/csv',
                         split_type='Line')
### END_SOLUTION

xgb_transformer.wait(logs=False)

In [ ]:
s3 = boto3.client('s3')
obj = s3.get_object(Bucket=bucket, Key="{}/batch-out/{}".format(prefix,'batch-in.csv.out'))
target_predicted = pd.read_csv(io.BytesIO(obj['Body'].read()),',',names=['class'])

def binary_convert(x):
    threshold = 0.5
    if x > threshold:
        return 1
    else:
        return 0

target_predicted_binary = target_predicted['class'].apply(binary_convert)


In [ ]:
plot_confusion_matrix(test['label'], target_predicted_binary)

In [ ]:
print_metrics(test['label'], target_predicted_binary)

Is the new model better or worse than the first model?

## 6. Tuning hyperparameters
([Go to top](#Lab-4.1:-Implementing-Sentiment-Analysis))

In this section, you will create a hyperparameter tuning job to tune the model.

__Note__: Tuning Hyperparameters takes about an hour to complete. If you don't have enough time, proceed to Section 7. You can also skip to section 7 after you start the tuning job, and return to its results later.

### Challenge: Creating the estimator to tune

The first step is to create an estimator to tune. Uncomment and complete the following estimator code:

In [ ]:
# xgb = sagemaker.estimator.Estimator(....)
### BEGIN_SOLUTION
xgb = sagemaker.estimator.Estimator(container,
                                    role=sagemaker.get_execution_role(), 
                                    instance_count= 1, # make sure you have limit set for these instances
                                    instance_type='ml.m5.2xlarge', 
                                    output_path=f's3://{bucket}/{prefix}/output',
                                    sagemaker_session=sagemaker.Session())
### END_SOLUTION

In [ ]:
xgb.set_hyperparameters(eval_metric='error',
                        objective='binary:logistic',
                        num_round=42,
                        silent=1)

### Challenge: Creating the hyperparameter ranges

Using the [XGBoost Tuning documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost-tuning.html), add hyperparameter ranges to the following cell.



In [ ]:
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner

hyperparameter_ranges = {'alpha': ContinuousParameter(0,1000)}

### BEGIN_SOLUTION
hyperparameter_ranges = {'alpha': ContinuousParameter(0, 1000),
                         'min_child_weight': ContinuousParameter(0, 120),
                         'subsample': ContinuousParameter(0.5, 1),
                         'eta': ContinuousParameter(0.1, 0.5),  
                         'num_round': IntegerParameter(1,4000)
                         }
### END_SOLUTION

### Challenge: Specifying the target metrics

Update the `objective_metric_name` and `objective_type` to appropriate values for a binary classification problem. For more information, see the [XGBoost Tuning documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost-tuning.html).

In [ ]:
objective_metric_name = '<INSERT_VALUE_HERE>'
objective_type = '<INSERT_VALUE_HERE>'

### BEGIN_SOLUTION
objective_metric_name = 'validation:error'
objective_type = 'Minimize'
### END_SOLUTION

Create the hyperparameter tuning job.

In [ ]:
tuner = HyperparameterTuner(xgb,
                            objective_metric_name,
                            hyperparameter_ranges,
                            max_jobs=10, # Set this to 10 or above depending upon budget & available time.
                            max_parallel_jobs=1,
                            objective_type=objective_type,
                            early_stopping_type='Auto',
                           )

Run the tuning job. Note that this job might take around 60 minutes to complete. 

In [ ]:
%%time
tuner.fit(inputs=data_channels, include_cls_metadata=False, wait=False)

If you want to try Section 7 while you wait, don't run the next cell and go to Section 7.

In [ ]:
tuner.wait()

When the tuning job is complete, you can view the metrics from the tuning jobs.

In [ ]:
from pprint import pprint
from sagemaker.analytics import HyperparameterTuningJobAnalytics

tuner_analytics = HyperparameterTuningJobAnalytics(tuner.latest_tuning_job.name, sagemaker_session=sagemaker.Session())

df_tuning_job_analytics = tuner_analytics.dataframe()

# Sort the tuning job analytics by the final metrics value
df_tuning_job_analytics.sort_values(
    by=['FinalObjectiveValue'],
    inplace=True,
    ascending=False if tuner.objective_type == "Maximize" else True)

# Show detailed analytics for the top 20 models
df_tuning_job_analytics.head(20)

## Using the best hyperparameter job

After the tuning job is complete, you can find the best tuning job from the **HyperparameterTuner** object.


In [ ]:
attached_tuner = HyperparameterTuner.attach(tuner.latest_tuning_job.name, sagemaker_session=sagemaker.Session())
best_training_job = attached_tuner.best_training_job()

In [ ]:
from sagemaker.estimator import Estimator
algo_estimator = Estimator.attach(best_training_job)

best_algo_model = algo_estimator.create_model(env={'SAGEMAKER_DEFAULT_INVOCATIONS_ACCEPT':"text/csv"})

Run the test data through the data processing pipeline so that you can test the model.

In [ ]:
%%time
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

text_features = ['text']
model_target = 'label'

text_processor_0 = Pipeline([
    ('text_vect_0', CountVectorizer(preprocessor=clean, max_features=500))
])

data_preprocessor = ColumnTransformer([
    ('text_pre_0', text_processor_0, text_features[0])
])

print('Datasets shapes before processing: ', train.shape, validate.shape, test.shape)
train_matrix = data_preprocessor.fit_transform(train)
test_matrix = data_preprocessor.transform(test)
validate_matrix = data_preprocessor.transform(validate)
print('Datasets shapes after processing: ', train_matrix.shape, validate_matrix.shape, test_matrix.shape)

Use the test data on a batch transformation by using the best algorithm from the hyperparameter tuning job.

In [ ]:
%%time
upload_s3_csv('batch-in.csv', 'batch-in', test_matrix, test, True)

batch_output = f's3://{bucket}/{prefix}/batch-out/'
batch_input = f's3://{bucket}/{prefix}/batch-in/{batch_X_file}'

xgb_transformer = best_algo_model.transformer(instance_count=1,
                                       instance_type='ml.m5.2xlarge',
                                       strategy='MultiRecord',
                                       assemble_with='Line',
                                       output_path=batch_output)
xgb_transformer.transform(data=batch_input,
                         data_type='S3Prefix',
                         content_type='text/csv',
                         split_type='Line')
xgb_transformer.wait(logs=False)

Process the results to calculate the class.

In [ ]:
s3 = boto3.client('s3')
obj = s3.get_object(Bucket=bucket, Key=f'{prefix}/batch-out/batch-in.csv.out')
target_predicted = pd.read_csv(io.BytesIO(obj['Body'].read()),',',names=['class'])

def binary_convert(x):
    threshold = 0.5
    if x > threshold:
        return 1
    else:
        return 0

target_predicted_binary = target_predicted['class'].apply(binary_convert)


Plot the confusion matrix and print the metrics.

In [ ]:
plot_confusion_matrix(test['label'], target_predicted_binary)

In [ ]:
print_metrics(test['label'], target_predicted_binary)

## 7. Using BlazingText
([Go to top](#Lab-4.1:-Implementing-Sentiment-Analysis))

In this section, you will use BlazingText, which is a built-in Amazon SageMaker algorithm. BlazingText can perform classification without modifications. You will reformat the data for BlazingText. You will then use the data to train the algorithm and compare the results to the previous models.


Start by obtaining the container for the algorithm.

In [ ]:
import sagemaker
from sagemaker.image_uris import retrieve

container = retrieve('blazingtext',boto3.Session().region_name,"latest")

Configure the prefixes for the training, validation, and test data.

In [ ]:
import io
    
prefix='lab41'
train_file='blazing_train.txt'
validate_file='blazing_validate.txt'
test_file='blazing_test.txt'

Remind yourself of what the data looks like.

In [ ]:
train.head()


BlazingText needs the data in the following format:

\__label__1 Caught this movie on the tube on a Sunday...

The following two cells convert the dataframes into the correct format, and upload them to Amazon S3.

In [ ]:
blazing_text_buffer = io.StringIO()
train.to_string(buf=blazing_text_buffer, columns=['label','text'], header=False, index=False, formatters=
                         {'label': '__label__{}'.format})
s3r = boto3.resource('s3')
s3r.Bucket(bucket).Object(os.path.join(prefix, 'blazing', train_file)).put(Body=blazing_text_buffer.getvalue())

In [ ]:
blazing_text_buffer = io.StringIO()
validate.to_string(buf=blazing_text_buffer, columns=['label','text'], header=False, index=False, formatters=
                         {'label': '__label__{}'.format})
s3r.Bucket(bucket).Object(os.path.join(prefix, 'blazing', validate_file)).put(Body=blazing_text_buffer.getvalue())

### Challenge: Training the BlazingText estimator

In the next cell, uncomment and complete the estimator code by specifying the missing values. 



In [ ]:
# bt_model = sagemaker.estimator.Estimator(container,
#                                         sagemaker.get_execution_role(), 
#                                         instance_count=, 
#                                         instance_type=,
#                                         volume_size = 30,
#                                         max_run = 360000,
#                                         input_mode= 'File',
#                                         output_path=,
#                                         sagemaker_session=

### BEGIN_SOLUTION
bt_model = sagemaker.estimator.Estimator(container,
                                         sagemaker.get_execution_role(), 
                                         instance_count=1, 
                                         instance_type='ml.c4.4xlarge',
                                         volume_size = 30,
                                         max_run = 360000,
                                         input_mode= 'File',
                                         output_path=s3_output_location,
                                         sagemaker_session=sagemaker.Session())

### END_SOLUTION

Use the following hyperparameters:

In [ ]:
bt_model.set_hyperparameters(mode="supervised",
                            epochs=10,
                            min_count=2,
                            learning_rate=0.05,
                            vector_dim=10,
                            early_stopping=True,
                            patience=4,
                            min_epochs=5,
                            word_ngrams=2)

Set up the training channel and the validate channel.

In [ ]:
train_channel = sagemaker.inputs.TrainingInput(
    f's3://{bucket}/{prefix}/blazing/{train_file}',
    content_type='text/csv')

validate_channel = sagemaker.inputs.TrainingInput(
    f's3://{bucket}/{prefix}/blazing/{validate_file}',
    content_type='text/csv')

data_channels_3 = {'train': train_channel, 'validation': validate_channel}

### Challenge: Starting the training job

Start the training job by entering the following code. (This step might take a few minutes.)

In [ ]:
%%time

### BEGIN_SOLUTION
bt_model.fit(inputs=data_channels_3, logs=False)
### END_SOLUTION

After the training job is complete, view the training metrics.

In [ ]:
sagemaker.analytics.TrainingJobAnalytics(bt_model._current_job_name, 
                                         metric_names = ['train:accuracy','validation:accuracy']
                                        ).dataframe()

In [ ]:
pd.options.display.max_rows
pd.set_option('display.max_colwidth', None)

Make a copy of the test data so that it can be formatted to use the model.

In [ ]:
bt_test = test.copy()
bt_test.head()

Format the dataset into the format that BlazingText needs.

In [ ]:
# bt_test['text'].str.strip()
bt_test.replace(r'\\n','', regex=True, inplace = True)
bt_test.rename(columns={'text':'source'}, inplace=True)
bt_test.drop(columns='label', inplace=True)

In [ ]:
print(bt_test.head().to_json(orient="records", lines=True))

Upload the file to Amazon S3.

In [ ]:
bt_file = 'bt_input.json'
blazing_text_buffer = io.StringIO()
bt_test.to_json(path_or_buf=blazing_text_buffer, orient="records", lines=True)

In [ ]:
s3r.Bucket(bucket).Object(os.path.join(prefix, 'blazing', bt_file)).put(Body=blazing_text_buffer.getvalue())


In [ ]:
batch_output = f's3://{bucket}/{prefix}/blazing/'
batch_input = f's3://{bucket}/{prefix}/blazing/{bt_file}'

Use a batch transformer on the test data. (This step might take a few minutes.)

In [ ]:
%%time
bt_transformer = bt_model.transformer(instance_count=1,
                                       instance_type='ml.m5.2xlarge',
                                       strategy='MultiRecord',
                                       assemble_with='Line',
                                       output_path=batch_output)

bt_transformer.transform(data=batch_input,
                         data_type='S3Prefix',
                         content_type='application/jsonlines',
                         split_type='Line')

bt_transformer.wait(logs=True)

Retrieve the results from Amazon S3.

In [ ]:
obj = s3.get_object(Bucket=bucket, Key=f'{prefix}/blazing/bt_input.json.out')

In [ ]:
target_predicted = pd.read_json(io.BytesIO(obj['Body'].read()),lines=True)

In [ ]:
target_predicted.head()

Reformat the results so that you can calculate the confusion matrix and the metrics.

In [ ]:
def binary_convert(label):
    label = label[0].replace('__label__','')
    return int(label)

target_predicted_binary = target_predicted['label'].apply(binary_convert)

In [ ]:
plot_confusion_matrix(test['label'], target_predicted_binary)

In [ ]:
print_metrics(test['label'], target_predicted_binary)

How did BlazingText perform compared to the previous models?

## 8. Using Amazon Comprehend
([Go to top](#Lab-4.1:-Implementing-Sentiment-Analysis))

In this section, you will use Amazon Comprehend to calculate the sentiment. Amazon Comprehend gives you positive and negative results, and it also shows neutral and mixed results. Amazon Comprehend is a managed service, and it requires less text processing before it's used. You won't need to process any text in this section.

Review what the data looks like in the `test` dataframe.

In [ ]:
test.head()

Using Amazon Comprehend can be as straightforward as making an API call.

The following cell outputs the first five results from Amazon Comprehend.

In [ ]:
import boto3
import json

comprehend = boto3.client(service_name='comprehend')
for n in range(5):
    text = test.iloc[n]['text']
    response = comprehend.detect_sentiment(Text=text, LanguageCode='en')
    sentiment = response['Sentiment']
    print(f'{sentiment} - {text}')


You can start a prediction job to process multiple items. The input must be formatted as a single input per line, and uploaded to Amazon S3. The text has a maximum size of 5120, so the `str.slice(0,5000)` function is used to trim long text.

In [ ]:
# Upload test file minus label to S3
def upload_comprehend_s3_csv(filename, folder, dataframe):
    csv_buffer = io.StringIO()
    
    dataframe.to_csv(csv_buffer, header=False, index=False )
    s3_resource.Bucket(bucket).Object(os.path.join(prefix, folder, filename)).put(Body=csv_buffer.getvalue())

comprehend_file = 'comprehend_input.csv'
upload_comprehend_s3_csv(comprehend_file, 'comprehend', test['text'].str.slice(0,5000))
test_url = f's3://{bucket}/{prefix}/comprehend/{comprehend_file}'
print(f'Uploaded input to {test_url}')

When the data is uploaded to Amazon S3, you start the job by using the `start_sentiment_detection_jon` function. 



### Challenge: Configuring the Amazon Comprehend job parameters

In the next cell, configure the Amazon Comprehend job parameters. 
- In __input_data_config__ - 
  - **S3Uri**: Replace *`<S3_INPUT_GOES_HERE>`* with the `test_uri` that was defined previously
  - **InputFormat**: Replace *`<INPUT_FORMAT_GOES_HERE>`* with `ONE_DOC_PER_LINE`
- In __output_data-config__ -  
  - **S3Uri**: Replace *`<S3_OUTPUT_GOES_HERE>`*  with the `s3_output_location`
  - **data_access_role_arn**: Replace *`arn:aws:iam::753256443787:role/service-role/c51302a798363l1767466t1w7-ComprehendDataAccessRole-396LD2R02VHP`* with the Amazon Resource Name (ARN) from the *Lab Details* file

In [ ]:
input_data_config={
    'S3Uri': 'S3_INPUT_GOES_HERE',
    'InputFormat': 'INPUT_FORMAT_GOES_HERE'
},

output_data_config={
    'S3Uri': 'S3_OUTPUT_GOES_HERE'
},
data_access_role_arn = 'arn:aws:iam::753256443787:role/service-role/c51302a798363l1767466t1w7-ComprehendDataAccessRole-396LD2R02VHP'

### BEGIN_SOLUTION
input_data_config={
    'S3Uri': test_url,
    'InputFormat': 'ONE_DOC_PER_LINE'
}
output_data_config={
    'S3Uri': s3_output_location
}
data_access_role_arn = 'arn:aws:iam::753256443787:role/service-role/c51302a798363l1767466t1w7-ComprehendDataAccessRole-396LD2R02VHP'
### END_SOLUTION

Now that you defined the job parameters, start the sentiment detection job.

In [ ]:
response = comprehend.start_sentiment_detection_job(
    InputDataConfig=input_data_config,
    OutputDataConfig=output_data_config,
    DataAccessRoleArn=data_access_role_arn,
    JobName='movie_sentiment',
    LanguageCode='en'
)

print(response['JobStatus'])

The following cell will loop until the job is completed. (This step might take a few minutes to complete.)

In [ ]:
%%time
import time
job_id = response['JobId']
while True:
    job_status=(comprehend.describe_sentiment_detection_job(JobId=job_id))
    if job_status['SentimentDetectionJobProperties']['JobStatus'] in ['COMPLETED','FAILED']:
        break            
    else:
        print('.', end='')
    time.sleep(15)
print((comprehend.describe_sentiment_detection_job(JobId=job_id))['SentimentDetectionJobProperties']['JobStatus'])

When the job is complete, you can return the details from the job by calling the `describe_sentiment_detection_job` function.

In [ ]:
output=(comprehend.describe_sentiment_detection_job(JobId=job_id))
print(output)

In the **OutputDataConfig** section, you should see the `S3Uri`. Extracting that URI will give you the file that you must download from Amazon S3. You can use the results to calculate metrics in the same way that you calculated the results from a batch transformation by using an algorithm.

In [ ]:
comprehend_output_file = output['SentimentDetectionJobProperties']['OutputDataConfig']['S3Uri']
comprehend_bucket, comprehend_key = comprehend_output_file.replace("s3://", "").split("/", 1)

s3r = boto3.resource('s3')
s3r.meta.client.download_file(comprehend_bucket, comprehend_key, 'output.tar.gz')

# Extract the tar file
import tarfile
tf = tarfile.open('output.tar.gz')
tf.extractall()

The extracted file should be named __output__. Read the the lines in this file.

In [ ]:
import json
data = ''
with open ('output', "r") as myfile:
    data = myfile.readlines()

Add the lines to an array.

In [ ]:
results = []
for line in data:
    json_data = json.loads(line)
    results.append([json_data['Line'],json_data['Sentiment']])

Convert the array to a pandas dataframe. 

In [ ]:
c = pd.DataFrame.from_records(results, index='index', columns=['index','sentiment'])
c.head()

The results contain **NEGATIVE**, **POSITIVE**, **NEUTRAL**, and **MIXED** results instead of numerical values. To compare these results to your test data, they can be mapped to numerical values, as shown in the following cell. The index in the returned results is also out of order. The `sort_index` function should fix this issue.

In [ ]:
class_mapper = {'NEGATIVE':0, 'POSITIVE':1, 'NEUTRAL':2, 'MIXED':3}
c['sentiment']=c['sentiment'].replace(class_mapper)
c = c.sort_index()
c.head()

In [ ]:
# Build list to compare for Amazon Comprehend
test_2 = test.reset_index()
test_3 = test_2.sort_index()
test_labels = test_3.iloc[:,2]

You can display a confusion matrix by using the `plot_confusion_matrix` function. Because Amazon Comprehend also includes __mixed__ and __neutral__ in the results, the chart will be different.

In [ ]:
plot_confusion_matrix(test_labels, c['sentiment'])

The existing function to print metrics won't work because you have too many data dimensions. The following code cell will calculate the same values.

In [ ]:
cm = confusion_matrix(test_labels, c['sentiment'])

TN = cm[0,0]
FP = cm[0,1]
FN = cm[1,0]
TP = cm[1,1]

Sensitivity  = float(TP)/(TP+FN)*100
# Specificity or true negative rate
Specificity  = float(TN)/(TN+FP)*100
# Precision or positive predictive value
Precision = float(TP)/(TP+FP)*100
# Negative predictive value
NPV = float(TN)/(TN+FN)*100
# Fall out or false positive rate
FPR = float(FP)/(FP+TN)*100
# False negative rate
FNR = float(FN)/(TP+FN)*100
# False discovery rate
FDR = float(FP)/(TP+FP)*100
# Overall accuracy
ACC = float(TP+TN)/(TP+FP+FN+TN)*100

print(f"Sensitivity or TPR: {Sensitivity}%")    
print(f"Specificity or TNR: {Specificity}%") 
print(f"Precision: {Precision}%")   
print(f"Negative Predictive Value: {NPV}%")  
print( f"False Positive Rate: {FPR}%") 
print(f"False Negative Rate: {FNR}%")  
print(f"False Discovery Rate: {FDR}%" )
print(f"Accuracy: {ACC}%") 

# Congratulations!

You have completed this lab, and you can now end the lab by following the lab guide instructions.

*©2021 Amazon Web Services, Inc. or its affiliates. All rights reserved. This work may not be reproduced or redistributed, in whole or in part, without prior written permission from Amazon Web Services, Inc. Commercial copying, lending, or selling is prohibited. All trademarks are the property of their owners.*
